In [2]:
import warnings
warnings.filterwarnings("ignore")

import functools, operator, requests, os, json
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END
from langchain.tools import tool
from typing import Annotated, Any, Dict, List, Optional, Sequence, TypedDict
from IPython.display import Markdown

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.tools.retriever import create_retriever_tool
from langchain_experimental.tools import PythonREPLTool

USER_AGENT environment variable not set, consider setting it to identify your requests.


#### Load environment variables

In [6]:
import os
from dotenv import load_dotenv
load_dotenv(dotenv_path=".env")

EMBEDDING_DEPLOYMENT_NAME = os.getenv('EMBEDDING_DEPLOYMENT_NAME')
CHATGPT_16K_DEPLOYMENT_NAME = os.getenv('CHATGPT_16K_DEPLOYMENT_NAME')
CHATGPT_TURBO_DEPLOYMENT_NAME = os.getenv('CHATGPT_TURBO_DEPLOYMENT_NAME')
GPT4_DEPLOYMENT_NAME = os.getenv('GPT4_DEPLOYMENT_NAME')
GPT4_TURBO_DEPLOYMENT_NAME = os.getenv('GPT4_TURBO_DEPLOYMENT_NAME')

API_KEY = os.getenv('southindia_API_KEY')
API_BASE = os.getenv('southindia_API_BASE')
API_TYPE = os.getenv('API_TYPE')
API_VERSION = os.getenv('API_VERSION')

#### Define LLM

#### OpenAI Style

In [7]:
# from langchain.chat_models import ChatOpenAI
# from langchain.embeddings.openai import OpenAIEmbeddings

# llm  = ChatOpenAI()
# embeddings = OpenAIEmbeddings()

#### Azure Style

In [8]:
from langchain.chat_models import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    azure_deployment=EMBEDDING_DEPLOYMENT_NAME,
    openai_api_key=API_KEY,
    azure_endpoint=API_BASE,
    openai_api_version=API_VERSION,
)

llm = AzureChatOpenAI(
        openai_api_base=API_BASE,
        openai_api_version=API_VERSION,
        deployment_name=GPT4_TURBO_DEPLOYMENT_NAME,
        openai_api_key=API_KEY,
        openai_api_type=API_TYPE,
        temperature=0)

/home/nalabsr1/venv/gen.ai_venv_39/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(


In [9]:
llm.invoke("What is 2+2 ?")

AIMessage(content='2 + 2 equals 4.', response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 14, 'total_tokens': 22}, 'model_name': 'gpt-4', 'system_fingerprint': 'fp_811936bd4f', 'finish_reason': 'stop', 'logprobs': None}, id='run-bfac7791-6db7-42f3-a552-89b9b30de4e3-0')

#### Agent Creation

In [10]:
import os
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from langchain.agents import initialize_agent
from langchain_community.retrievers import TavilySearchAPIRetriever
from langchain.agents import load_tools

def get_tavily_search_retriever():
    retriever = TavilySearchAPIRetriever(k=7)
    retriever_description = "Use this tool when you need to answer questions related to current events and latest happenings or anything related to the user ask"
    retrieve_tool = create_retriever_tool(retriever=retriever, name="google_search", description=retriever_description)
    return retrieve_tool

tavily_search_tool = get_tavily_search_retriever()
tools = load_tools(["llm-math"], llm=llm)
tools = tools + [tavily_search_tool]

# Initialize memory
memory = ConversationBufferMemory(memory_key="chat_history")

# Initialize the agent with verbose output
agent_chain = initialize_agent(tools, llm, agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory)

# Run the agent with a sample input
# response = agent_chain.run(input="What is the sum of temperatures of hyderbad and guntur today?")
# print(response)

/home/nalabsr1/venv/gen.ai_venv_39/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [11]:
agent_chain.run(input="Which is hotter? Hyderabad or Basel?")

/home/nalabsr1/venv/gen.ai_venv_39/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: google_search
Action Input: current temperature in Hyderabad
Observation: Get the latest weather information for Hyderabad, Telangana, India, including temperature, humidity, wind, pressure, and precipitation. See hourly and daily forecasts, radar maps, and severe weather alerts.

Get the current temperature and outlook for Hyderabad, Telangana, India. See hourly and 10-day forecasts, radar maps, air quality, and allergy outlook for Hyderabad.

Get the latest weather forecast for Hyderabad, including warnings, temperature, rain, wind and visibility. 14-day weather forecast for Hyderabad.

Everything you need to know about today's weather in Hyderabad, Telangana, India. High/Low, Precipitation Chances, Sunrise/Sunset, and today's Temperature History.

Current Weather Hyderabad . 26 o C. 79%. West-southwesterly 11.1 km/h; Observation Time: 17 Jul, 22.30 IST . Sunrise:05:51 (IST) Sunset: 18:54 (IST) Moon

"Based on the current weather information available, Hyderabad has a temperature of 26°C, while Basel has a temperature of 78°F, which is approximately 25.6°C. The temperatures are quite similar, but Hyderabad is slightly hotter than Basel at the moment. Please note that weather conditions can change rapidly, and for the most up-to-date information, it's always best to check a reliable weather source."